In [ ]:
import imageio.v2 as iio

import numpy as np
from PIL import Image
from scipy.ndimage import map_coordinates
import os
os.environ["IMAGEIO_FFMPEG_EXE"] = '/usr/bin/ffmpeg'
from tqdm import tqdm

def map_to_sphere(x, y, z, yaw_radian, pitch_radian, roll_radian):
    # Apply roll rotation (rotate x,y around z-axis)
    x_rolled = x * np.cos(roll_radian) - y * np.sin(roll_radian)
    y_rolled = x * np.sin(roll_radian) + y * np.cos(roll_radian)
    
    theta = np.arccos(z / np.sqrt(x_rolled ** 2 + y_rolled ** 2 + z ** 2))
    phi = np.arctan2(y_rolled, x_rolled)

    theta_prime = np.arccos(np.sin(theta) * np.sin(phi) * np.sin(pitch_radian) +
                            np.cos(theta) * np.cos(pitch_radian))

    phi_prime = np.arctan2((np.sin(theta) * np.sin(phi) * np.cos(pitch_radian) -
                            np.cos(theta) * np.sin(pitch_radian)),
                           np.sin(theta) * np.cos(phi))

    phi_prime += yaw_radian
    phi_prime = phi_prime % (2 * np.pi)

    return theta_prime, phi_prime

def interpolate_color(coords, img, method='bilinear'):
    order = {'nearest': 0, 'bilinear': 1, 'bicubic': 3}.get(method, 1)
    red = map_coordinates(img[:, :, 0], coords, order=order, mode='reflect')
    green = map_coordinates(img[:, :, 1], coords, order=order, mode='reflect')
    blue = map_coordinates(img[:, :, 2], coords, order=order, mode='reflect')
    return np.stack((red, green, blue), axis=-1)


def panorama_to_plane(panorama_path, FOV, output_size, yaw, pitch, roll=90):
    panorama = Image.open(panorama_path).convert('RGB')
    pano_width, pano_height = panorama.size
    pano_array = np.array(panorama)
    yaw_radian = np.radians(yaw)
    pitch_radian = np.radians(pitch)
    roll_radian = np.radians(roll)

    W, H = output_size
    f = (0.5 * W) / np.tan(np.radians(FOV) / 2)

    u, v = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')

    x = u - W / 2
    y = H / 2 - v
    z = f

    theta, phi = map_to_sphere(x, y, z, yaw_radian, pitch_radian, roll_radian)   
    U = phi * pano_width / (2 * np.pi)
    V = theta * pano_height / np.pi

    U, V = U.flatten(), V.flatten()
    coords = np.vstack((V, U))

    colors = interpolate_color(coords, pano_array)
    output_image = Image.fromarray(colors.reshape((H, W, 3)).astype('uint8'), 'RGB')

    return output_image


In [ ]:
import os
from groundedvision.config import RAW_DATA_DIR, PROCESSED_DATA_DIR
from tqdm import tqdm

from loguru import logger

In [ ]:
path = f'{PROCESSED_DATA_DIR}/matched_pairs_json'
list_of_aligned_directories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

In [ ]:
from pathlib import Path

for folder_path in list_of_aligned_directories:
    input_folder_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}"
    for image_path in os.listdir(input_folder_path):
        output_path = f"{PROCESSED_DATA_DIR}/matched_pairs_json/{folder_path}"
        full_path = os.path.join(input_folder_path, image_path)
        path_full_path = Path(full_path)
        # check if the image patch contains
        if "_aligned.JPG" in full_path:
            logger.info(f"Processing {full_path}")
            writer = iio.get_writer(f'{input_folder_path}/{path_full_path.stem}.mp4', format='FFMPEG', mode='I', fps=5)
            for deg in tqdm(np.arange(0, 360, 10)):
                output_image = panorama_to_plane(full_path, 90, (600, 600), deg, 90, 0)
                writer.append_data(np.array(output_image))
            writer.close()